# Data Analysis with Python
_T.J. Langford_  
_Wright Lab & YCRC_  
_April 9, 2019_  

# Overview

- Data Processing with `numpy`
- Data Analysis with `pandas`
- Data Visualization with `matplotlib`

# Tools and Requirements

- Language: Python 3.6
- Modules: `numpy`, `pandas`, `matplotlib`
- Jupyter notebook

# Comment: Python 2 versus 3
- Major modules will be dropping Python2 support in 2019
- Worth transitioning to get new features
- This tutorial uses Python3
- see https://wiki.python.org/moin/Python2orPython3

# Github Repository

- The materials from this tutorial are available on GitHub at [link](...)
- Can also launch an active version in a Binder environment here: